# Yaw Angle Pattern Classifier

## Overview

This notebook implements a machine learning-based classifier designed to detect specific patterns in the yaw angle data of human head movements extracted from video frames. The classifier identifies whether the person in the video is rotating their head from one side to the other based on the yaw angles predicted by a Convolutional Neural Network (CNN) model.

## Workflow

### 1. Data Collection

The yaw angles are predicted for each frame of a video using a pre-trained CNN model. The output is a map where:
- **Key**: Index given by the minute and second of the frame.
- **Value**: A numeric value representing the yaw angle of the face.

Example output:
```json
{
  "0.2": "-90",
  "0.4": "-85",
  "0.6": "-60",
  "0.8": "-45",
  ...
  "3.0": "65",
  "3.2": "75",
  "3.4": "87",
  "3.6": "90"
}
```

### Features
* __Number of Peaks__: Count the number of local maxima in the sequence.
* __Amplitude of Peaks and Troughs__: Measure the difference between the maximum and minimum yaw angles.
* __Average Yaw__: Compute the mean yaw angle across the sequence.
* __Frequency of Peaks__: Calculate the frequency at which peaks occur.
* __Yaw Range__: Calculate the range (maximum - minimum) of yaw angles.
* __Standard Deviation__: Compute the standard deviation of yaw angles to capture the variability.


## Import the neccesary dependencies

In [2]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Extract example features

In [3]:
def getFeatureVectorWithLabel(yaw_df):
    yaw_values = yaw_df.iloc[:, :-1].to_numpy()
    labs = yaw_df["label"]
    
    num_peaks_list = []
    num_troughs_list = []
    amplitude_peaks_list = []
    average_yaw_list = []
    yaw_range_list = []
    std_yaw_list = []
    labels = []

    for i, row in enumerate(yaw_values):
        
        peaks, _ = find_peaks(row)
        troughs, _ = find_peaks(-row)

        num_peaks = len(peaks)
        num_troughs = len(troughs)
        amplitude_peaks = row[peaks].max() - row[troughs].min() if peaks.size > 0 and troughs.size > 0 else np.nan
        average_yaw = np.mean(row)
        yaw_range = np.ptp(row)  # Peak-to-peak range
        std_yaw = np.std(row)

        num_peaks_list.append(num_peaks)
        num_troughs_list.append(num_troughs)
        amplitude_peaks_list.append(amplitude_peaks)
        average_yaw_list.append(average_yaw)
        yaw_range_list.append(yaw_range)
        std_yaw_list.append(std_yaw)
        labels.append(labs.iloc[i])

    df = pd.DataFrame({
        'num_peaks': num_peaks_list,
        'num_troughs': num_troughs_list,
        'amplitude_peaks': amplitude_peaks_list,
        'average_yaw': average_yaw_list,
        'yaw_range': yaw_range_list,
        'std_yaw': std_yaw_list,
        'label': labels
    })
    
    return df


def getFeatureVector(yaw_df):
    yaw_values = yaw_df.iloc[:, :-1].to_numpy()
    
    num_peaks_list = []
    num_troughs_list = []
    amplitude_peaks_list = []
    average_yaw_list = []
    yaw_range_list = []
    std_yaw_list = []

    for i, row in enumerate(yaw_values):
        
        peaks, _ = find_peaks(row)
        troughs, _ = find_peaks(-row)

        num_peaks = len(peaks)
        num_troughs = len(troughs)
        amplitude_peaks = row[peaks].max() - row[troughs].min() if peaks.size > 0 and troughs.size > 0 else np.nan
        average_yaw = np.mean(row)
        yaw_range = np.ptp(row)  # Peak-to-peak range
        std_yaw = np.std(row)

        num_peaks_list.append(num_peaks)
        num_troughs_list.append(num_troughs)
        amplitude_peaks_list.append(amplitude_peaks)
        average_yaw_list.append(average_yaw)
        yaw_range_list.append(yaw_range)
        std_yaw_list.append(std_yaw)

    df = pd.DataFrame({
        'num_peaks': num_peaks_list,
        'num_troughs': num_troughs_list,
        'amplitude_peaks': amplitude_peaks_list,
        'average_yaw': average_yaw_list,
        'yaw_range': yaw_range_list,
        'std_yaw': std_yaw_list
    })
    
    return df


In [4]:
yaw_data = {
  "0.2": 2.5284867,
  "0.4": 2.749594,
  "0.6000000000000001": 2.1379437,
  "0.8": 1.7823929,
  "1.0": 1.6482912,
  "1.2000000000000002": 5.0864453,
  "1.4000000000000001": 14.992984,
  "1.6": 16.431183,
  "1.8": 16.039757,
  "2.0": 16.063456,
  "2.2": 12.145712,
  "2.4000000000000004": 1.3173454,
  "2.6": -1.07616,
  "2.8000000000000003": -0.4528833,
  "3.0": 0.51127344,
  "3.2": 13.553554,
  "3.4000000000000004": 19.458012,
  "3.6": 18.421278,
  "3.8000000000000003": 22.45929,
  "4.0": 22.07545,
  "4.2": 22.521818,
  "4.4": 23.279245,
  "4.6000000000000005": 21.392345,
  "4.800000000000001": 19.092186,
  "5.0": 14.48045,
  "5.2": -4.107289
}

yaw_values = np.array(list(yaw_data.values()))
peaks, _ = find_peaks(yaw_values)
troughs, _ = find_peaks(-yaw_values)
num_peaks = len(peaks)
num_troughs = len(troughs)
amplitude_peaks = yaw_values[peaks].max() - yaw_values[troughs].min()
average_yaw = np.mean(yaw_values)
yaw_range = np.ptp(yaw_values)
std_yaw = np.std(yaw_values)

features = [num_peaks, num_troughs, amplitude_peaks, average_yaw, yaw_range, std_yaw]

print("Feature vector:", features)

Feature vector: [6, 5, 24.355405, 10.943544628461542, 27.386533999999997, 8.962988566088272]


## Build the classyfier

In [8]:
dataset = pd.read_csv('../dataset/combined.csv')

data = getFeatureVectorWithLabel(dataset)
features_dataset = data[["num_peaks", "num_troughs", "amplitude_peaks", "average_yaw", "yaw_range", "std_yaw"]]
labels = data["label"]

X_train, X_test, y_train, y_test = train_test_split(features_dataset, labels, test_size=0.3, random_state=42)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


## Test the model

In [10]:
new_yaw_data = {
    0.2: 3.656726, 0.4: 3.1144602, 0.6000000000000001: 3.546911, 0.8: 3.6577039, 
    1.0: 11.964856, 1.2000000000000002: 17.873423, 1.4000000000000001: 18.166483, 
    1.6: 18.349371, 1.8: 19.971073, 2.0: 16.77458, 2.2: 21.979391, 
    2.4000000000000004: 19.965542, 2.6: 19.677303, 2.8000000000000003: 6.381784, 
    3.0: 9.820981, 3.2: 8.400116, 3.4000000000000004: 8.426596, 
    3.6: 6.2372966, 3.8000000000000003: 7.950895, 4.0: 6.4219823, 
    4.2: 6.394048, 4.4: 5.8880568, 4.6000000000000005: 5.868197, 
    4.800000000000001: 6.2909064, 5.0: 1.6750643, 5.2: 7.3736625, 
    5.4: 17.891846, 5.6000000000000005: 19.232447, 5.800000000000001: 22.017963, 
    6.0: 17.674093, 6.2: 16.985455, 6.4: 16.376997, 6.6000000000000005: 16.396362, 
    6.800000000000001: 16.338333, 7.0: 17.114092, 7.2: 8.680025, 
    7.4: 9.172808, 7.6000000000000005: 9.178544, 7.800000000000001: 7.429461, 
    8.0: 8.059247, 8.200000000000001: 7.59325, 8.4: 8.45055, 8.6: 8.174861, 
    8.8: 8.502427, 9.0: 8.41412, 9.200000000000001: 4.6104126, 
    9.4: 1.7861786, 9.600000000000001: 4.8285484, 9.8: 6.523864, 10.0: 4.684122
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame([yaw_data])
data = getFeatureVector(df)

prediction = clf.predict(data)
prediction[0]

1